In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.layers import BatchNormalization

es = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

EPOCHS = 100

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

root_path = "/usr/share/MSI25/data_images/"

gen = ImageDataGenerator(
    rescale=1./255,
#     horizontal_flip=True,
#     vertical_flip=True,
#     rotation_range=20,
    validation_split=0.2
)

# Load Data
train_dataset = gen.flow_from_directory(
    root_path,
    class_mode="categorical",
    classes=["smile","non_smile"],
    shuffle=True,
    batch_size=32,
    target_size=(64,64),
    subset="training"
)

validation_dataset = gen.flow_from_directory(
    root_path,
    class_mode="categorical",
    classes=["smile","non_smile"],
    shuffle=True,
    batch_size=32,
    target_size=(64,64),
    subset="validation"
)

In [ ]:
vgg_model = Sequential()

pretrained_model= tf.keras.applications.VGG16(include_top=False,
                   input_shape=(64,64,3),
                   pooling='avg',classes=2,
                   weights='imagenet') 
for layer in pretrained_model.layers: #kluczowe
        layer.trainable=False

vgg_model.add(pretrained_model)

vgg_model.add(Flatten())
vgg_model.add(Dense(2, activation='softmax'))
vgg_model.summary()

In [ ]:
vgg_model.compile(optimizer="adam", loss='categorical_crossentropy',metrics=['accuracy'])

history = vgg_model.fit(train_dataset, validation_data=validation_dataset, epochs=EPOCHS, callbacks = [es])

In [ ]:
hist = pd.DataFrame(history.history)
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(hist.loss, label = "loss")
plt.plot(hist.val_loss, label = "val_loss")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.legend()

plt.subplot(1,2,2)
plt.plot(hist.accuracy, label = "acc")
plt.plot(hist.val_accuracy, label = "val_acc")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.legend()

In [ ]:
# Znajdz w internecie listę dostępnych modeli i sprawdź ich skuteczność